## Fish No Fish CONVNET WOOPWOOP
I love buzzwords.


In [48]:
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from keras.regularizers import l2, activity_l2
from keras.utils import np_utils
import cv2

%matplotlib inline

In [39]:
%load_ext autoreload
%autoreload 2
import sys
paths = ['../models', '../dataset', '../util']
for path in paths:
    if path not in sys.path:
        sys.path.append(path)
 
from bounding_boxes import *
from data_generators import get_data
import dataset_FishNoFish as dataset
from callbacks import ShowSegmentation

bboxes = get_bounding_boxes('../data')
bboxes = largest_bbox_per_image(bboxes)
train_data = dataset.train
val_data = dataset.validation
 
train_gen = get_data(train_data, '../../Data/train', dataset.labels)
val_gen = get_data(val_data, '../../Data/train', dataset.labels)
n_labels = len(dataset.labels)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Function to define the layers of the convnet.

In [60]:
def keras_model_conv():
    model = Sequential()
    
    model.add(BatchNormalization(axis=1,  input_shape=(3, 256, 256)))
    model.add(Convolution2D(16, 3, 3, border_mode='same', activation='relu',))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.50))

    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.50))
    
    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.50))
    
    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(BatchNormalization(axis=1))
    
    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(BatchNormalization(axis=1))
    
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(BatchNormalization(axis=1))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu', W_regularizer=l2(1e-6)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

    return model

Initialization of the model.

In [61]:
model = keras_model_conv()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_120 (BatchNor (None, 3, 256, 256)   12          batchnormalization_input_18[0][0]
____________________________________________________________________________________________________
convolution2d_103 (Convolution2D (None, 16, 256, 256)  448         batchnormalization_120[0][0]     
____________________________________________________________________________________________________
batchnormalization_121 (BatchNor (None, 16, 256, 256)  64          convolution2d_103[0][0]          
____________________________________________________________________________________________________
maxpooling2d_69 (MaxPooling2D)   (None, 16, 128, 128)  0           batchnormalization_121[0][0]     
___________________________________________________________________________________________

Fit the model using a generator.

Use: 
fit_generator(self, generator, samples_per_epoch, nb_epoch, verbose=1, callbacks=None, validation_data=None, nb_val_samples=None, class_weight=None, max_q_size=10, nb_worker=1, pickle_safe=False, initial_epoch=0)


In [62]:
model.fit_generator(train_gen, samples_per_epoch=len(train_data),
                    nb_epoch=100, verbose=1,
                    callbacks=None,
                    validation_data=val_gen, nb_val_samples=len(val_data))

Epoch 1/100
768/761 [==============================] - 18s - loss: 1.5503 - acc: 0.7383 - val_loss: 1.6251 - val_acc: 0.4531

In [63]:
model.fit_generator(train_gen, samples_per_epoch=len(train_data),
                    nb_epoch=50, verbose=1,
                    callbacks=None,
                    validation_data=val_gen, nb_val_samples=len(val_data))

Epoch 1/50
768/761 [==============================] - 18s - loss: 0.0809 - acc: 0.9948 - val_loss: 0.5453 - val_acc: 0.9167

In [68]:
imgs = next(val_gen)
for img in imgs:
    print model.predict(img)


[[  1.00000000e+00   6.01630343e-11]
 [  1.00000000e+00   7.71212964e-16]
 [  1.00000000e+00   2.27480174e-19]
 [  1.00000000e+00   3.80479407e-29]
 [  1.00000000e+00   2.66698861e-14]
 [  5.10872967e-30   1.00000000e+00]
 [  4.14026408e-06   9.99995828e-01]
 [  1.00000000e+00   0.00000000e+00]
 [  4.35404974e-22   1.00000000e+00]
 [  0.00000000e+00   1.00000000e+00]
 [  9.99999046e-01   9.05143963e-07]
 [  8.33842124e-26   1.00000000e+00]
 [  1.00000000e+00   2.64571643e-36]
 [  6.51432711e-15   1.00000000e+00]
 [  1.00000000e+00   1.05991983e-20]
 [  1.00000000e+00   2.36033854e-17]
 [  1.00000000e+00   3.00039534e-14]
 [  1.00000000e+00   6.23209774e-12]
 [  8.21856840e-29   1.00000000e+00]
 [  0.00000000e+00   1.00000000e+00]
 [  7.88613966e-11   1.00000000e+00]
 [  1.00000000e+00   2.38215236e-25]
 [  1.00000000e+00   1.97174584e-35]
 [  1.43536356e-08   1.00000000e+00]
 [  1.00000000e+00   1.99470498e-22]
 [  1.00000000e+00   1.50818735e-31]
 [  4.58278055e-13   1.00000000e+00]
 

ValueError: Error when checking : expected batchnormalization_input_18 to have 4 dimensions, but got array with shape (32L, 2L)

In [64]:
model.save('fish_nofishmodel.h5')